
# Building Multi-Agent Systems with Strands Agent Graph
Multi-agent systems leverage multiple specialized AI agents working together to solve complex problems through coordinated collaboration. Each agent has specific capabilities and roles, connected through explicit communication pathways.

In this lab, you'll learn to build multi-agent systems using the Strands Agent Graph tool. We'll progress from basic concepts to advanced implementations, exploring different topologies and real-world applications.

**Learning Objectives:**
By the end of this notebook, you'll be able to:
- Understand the three core components of agent graphs (nodes, edges, topologies)
- Create and manage different topology patterns (star, mesh, hierarchical)
- Implement both natural language and structured approaches to agent graphs
- Send targeted messages between specific agents
- Monitor and control multi-agent networks
- Design specialized agent systems for real-world scenarios

## Prerequisites

- Python 3.10+
- AWS account with Anthropic Claude 3.7 enabled on Amazon Bedrock
- IAM role with permissions to use Amazon Bedrock
- Basic understanding of AI agents and prompt engineering

## Setup and Installation

Before we start, let's install the requirement packages for `strands-agents` and `strands-agents-tools`

In [ ]:
!pip install -r requirements.txt

### Importing required packages

Next we can import the required packages

In [ ]:
import boto3
from strands import Agent
from strands_tools import agent_graph
import json

## Understanding Agent Graph Components
An agent graph is a structured network of interconnected AI agents designed to solve complex problems through coordinated collaboration. Each agent represents a specialized node with specific capabilities, and the connections between agents define explicit communication pathways.

Before we start building, let's understand the three primary components of an agent graph:

### 1. Nodes (Agents)
Each node represents an AI agent with:
- **Identity**: Unique identifier within the graph
- **Role**: Specialized function or purpose
- **System Prompt**: Instructions defining the agent's behavior
- **Tools**: Capabilities available to the agent
- **Message Queue**: Buffer for incoming communications

### 2. Edges (Connections)
Edges define communication pathways with:
- **Direction**: One-way or bidirectional information flow
- **Relationship**: How agents relate (supervisor/worker, peer-to-peer)
- **Message Passing**: Mechanism for transferring information

### 3. Topology Patterns
Different network structures for different use cases:
- **Star**: A central coordinator with radiating specialists, ideal for centralized workflows like content creation with editorial oversight or customer service with escalation paths.
- **Mesh**: Fully connected network where all agents can communicate directly with each other, ideal for collaborative problem-solving, debates, and consensus-building.
- **Hierarchical**: Tree structure with parent-child relationships, ideal for layered processing, project management with task delegation, and multi-level review processes.

![topologies](images/topologies.png)

### Creating your First Agent Graph - Star Topology

Let's start with the star topology, ideal for centralized workflows with editorial oversight. We will build a research agent with the team leader being the coordinator agent and a data analyst and a domain expert being the specialist agents. Architecture looks as following:

<div style="text-align:left">
    <img src="images/star_architecture.png" width="55%" />
</div>

In [ ]:
# Step 1: Initialize an agent with agent_graph capability
coordinator_agent = Agent(tools=[agent_graph])

# Step 2: Create a structured agent graph using the agent_graph tool
result = coordinator_agent.tool.agent_graph(
    action="create",
    graph_id="research_team",
    topology={
        "type": "star",
        "nodes": [
            {
                "id": "coordinator",
                "role": "team_lead",
                "system_prompt": """You are a research team leader coordinating specialists.
                provide a short analysis, no need for follow ups"""
            },
            {
                "id": "data_analyst",
                "role": "analyst",
                "system_prompt": """You are a data analyst specializing in statistical analysis.
                provide a short analysis, no need for follow ups"""
            },
            {
                "id": "domain_expert",
                "role": "expert",
                "system_prompt": """You are a domain expert with deep subject knowledge.
                provide a short analysis, no need for follow ups"""
            }
        ],
        "edges": [
            {"from": "coordinator", "to": "data_analyst"},
            {"from": "coordinator", "to": "domain_expert"},
            {"from": "data_analyst", "to": "coordinator"},
            {"from": "domain_expert", "to": "coordinator"}
        ]
    }
)

print("Research team created successfully!")
print(f"Result: {result}")

# Step 3: Check the status of our newly created network
status_result = coordinator_agent.tool.agent_graph(
    action="status",
    graph_id="research_team"
)

print("Network Status:")
print(status_result)

# Step 4: Send a task to the coordinator
print("Sending task to the coordinator...")

message_result = coordinator_agent.tool.agent_graph(
    action="message",
    graph_id="research_team",
    message={
        "target": "coordinator",
        "content": """Analyze the impact of remote work on employee productivity.
        provide a short analysis, no need for follow ups"""
    }
)

print("Task sent to coordinator!")
print(f"Response: {message_result}")
print("============================================================")
print("============================================================")

# Step 5: Direct communication with specialists
print("Sending direct message to data analyst...")

analyst_result = coordinator_agent.tool.agent_graph(
    action="message",
    graph_id="research_team",
    message={
        "target": "data_analyst",
        "content": """What are the key productivity metrics we should 
        track for remote work analysis? provide a short analysis, no need for follow ups"""
    }
)

print(f"Data Analyst Response: {analyst_result}")
print("============================================================")
print("============================================================")

# Step 6: List all active networks
print("Checking all active agent networks...")

list_result = coordinator_agent.tool.agent_graph(action="list")
print(f"Active Networks: {list_result}")
print("============================================================")
print("============================================================")
# Manage the network
coordinator_agent("Shut down the content research_team when they're done")

### Creating a Mesh Topology - Collaborative Problem-Solving

Now let's create a mesh topology where all agents can communicate directly with each other, ideal for debates and consensus-building. We will create a decision commitee agent with a financial advisor, technical architect, market researcher and risk analyst. The architecture looks as following:
<div style="text-align:left">
    <img src="images/mesh_architecture.png" width="55%" />
</div>

In [ ]:
# Step 1: Create a mesh topology for collaborative decision-making
print("🕸️ Creating mesh topology for collaborative analysis...")
mesh_agent = Agent(tools=[agent_graph])
mesh_result = mesh_agent.tool.agent_graph(
    action="create",
    graph_id="decision_committee",
    topology={
        "type": "mesh",
        "nodes": [
            {
                "id": "financial_advisor",
                "role": "finance_expert",
                "system_prompt": "You are a financial advisor focused on cost-benefit analysis, budget implications, and ROI calculations. Engage with other experts to build comprehensive financial perspectives."
            },
            {
                "id": "technical_architect",
                "role": "tech_expert", 
                "system_prompt": "You are a technical architect who evaluates feasibility, implementation challenges, and technical risks. Collaborate with other experts to ensure technical viability."
            },
            {
                "id": "market_researcher",
                "role": "market_expert",
                "system_prompt": "You are a market researcher who analyzes market conditions, user needs, and competitive landscape. Work with other experts to validate market opportunities."
            },
            {
                "id": "risk_analyst",
                "role": "risk_expert",
                "system_prompt": "You are a risk analyst who identifies potential risks, mitigation strategies, and compliance issues. Collaborate with other experts to ensure comprehensive risk assessment."
            }
        ],
        "edges": [
            # Full mesh - everyone can talk to everyone
            {"from": "financial_advisor", "to": "technical_architect"},
            {"from": "financial_advisor", "to": "market_researcher"},
            {"from": "financial_advisor", "to": "risk_analyst"},
            {"from": "technical_architect", "to": "financial_advisor"},
            {"from": "technical_architect", "to": "market_researcher"},
            {"from": "technical_architect", "to": "risk_analyst"},
            {"from": "market_researcher", "to": "financial_advisor"},
            {"from": "market_researcher", "to": "technical_architect"},
            {"from": "market_researcher", "to": "risk_analyst"},
            {"from": "risk_analyst", "to": "financial_advisor"},
            {"from": "risk_analyst", "to": "technical_architect"},
            {"from": "risk_analyst", "to": "market_researcher"}
        ]
    }
)

print("Mesh topology decision committee created!")
print(f"Result: {mesh_result}")
print("============================================================")
print("============================================================")

# Step 2: Present a complex decision to the committee
print("Presenting decision scenario to the committee...")

# Send to financial advisor first
financial_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "financial_advisor",
        "content": "Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?"
    }
)

print(f"Financial Advisor: {financial_response}")
print("============================================================")
print("============================================================")

# Step 3: Get technical perspective
technical_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "technical_architect",
        "content": "Regarding the AI customer service platform proposal - what are the key technical considerations, infrastructure requirements, and implementation challenges?"
    }
)

print(f"Technical Architect: {technical_response}")
print("============================================================")
print("============================================================")

# Step 4: Gather market insights
market_response = mesh_agent.tool.agent_graph(
    action="message",
    graph_id="decision_committee",
    message={
        "target": "market_researcher",
        "content": "For the AI customer service platform - what does the market landscape look like? Who are our competitors and what's the demand?"
    }
)

print(f"Market Researcher: {market_response}")
print("============================================================")
print("============================================================")
# Manage the network
mesh_agent("Shut down the content decision_committee when they're done")

### Creating a Hierarchical Topology - Organizational Structure

Finally, let's create a hierarchical topology that mimics organizational structures with clear reporting lines.

We will create a consulting firm agent with a hierarchical structure. In our team, we will have a managing partner, a strategy head, an operations head, a strategy analyst, an operations analyst and a data specialist. The architecture looks as following:

<div style="text-align:left">
    <img src="images/hierarchical_architecture.png" width="55%" />
</div>

In [ ]:
# Step 1: Create hierarchical topology
print("Creating hierarchical topology for organizational analysis...")

hierarchical_agent = Agent(tools=[agent_graph])
hierarchy_result = hierarchical_agent.tool.agent_graph(
    action="create",
    graph_id="consulting_firm",
    topology={
        "type": "hierarchical",
        "nodes": [
            {
                "id": "managing_partner",
                "role": "executive",
                "system_prompt": "You are a managing partner who oversees all consulting engagements. Delegate tasks to department heads, synthesize their reports, and provide executive-level recommendations to clients."
            },
            {
                "id": "strategy_head",
                "role": "department_manager",
                "system_prompt": "You are the head of strategy consulting. Manage strategic analysis projects, coordinate with junior consultants, and report findings to the managing partner."
            },
            {
                "id": "operations_head",
                "role": "department_manager", 
                "system_prompt": "You are the head of operations consulting. Oversee operational efficiency projects, manage your team, and provide operational insights to leadership."
            },
            {
                "id": "strategy_analyst",
                "role": "junior_consultant",
                "system_prompt": "You are a junior strategy consultant specializing in market analysis, competitive intelligence, and strategic planning. Report to the strategy head."
            },
            {
                "id": "operations_analyst",
                "role": "junior_consultant",
                "system_prompt": "You are a junior operations consultant focusing on process optimization, efficiency analysis, and operational improvements. Report to the operations head."
            },
            {
                "id": "data_specialist",
                "role": "junior_consultant",
                "system_prompt": "You are a data specialist supporting both strategy and operations with quantitative analysis, modeling, and data-driven insights."
            }
        ],
        "edges": [
            # Managing partner to department heads
            {"from": "managing_partner", "to": "strategy_head"},
            {"from": "managing_partner", "to": "operations_head"},
            {"from": "strategy_head", "to": "managing_partner"},
            {"from": "operations_head", "to": "managing_partner"},
            
            # Department heads to analysts
            {"from": "strategy_head", "to": "strategy_analyst"},
            {"from": "strategy_head", "to": "data_specialist"},
            {"from": "operations_head", "to": "operations_analyst"},
            {"from": "operations_head", "to": "data_specialist"},
            
            # Analysts to department heads
            {"from": "strategy_analyst", "to": "strategy_head"},
            {"from": "operations_analyst", "to": "operations_head"},
            {"from": "data_specialist", "to": "strategy_head"},
            {"from": "data_specialist", "to": "operations_head"}
        ]
    }
)

print("Hierarchical consulting firm created!")
print(f"Result: {hierarchy_result}")
print("============================================================")
print("============================================================")

# Step 2: Send a complex engagement to the managing partner
print("Sending client engagement to managing partner...")

engagement_response = hierarchical_agent.tool.agent_graph(
    action="message",
    graph_id="consulting_firm",
    message={
        "target": "managing_partner",
        "content": "We have a new client engagement: A mid-size manufacturing company wants to expand internationally while improving operational efficiency. They need both strategic market entry advice and operational optimization. Please coordinate a comprehensive analysis."
    }
)

print(f"Managing Partner Response: {engagement_response}")
print("============================================================")
print("============================================================")

# Step 3: Check how the work flows down the hierarchy
print("Checking the status of our consulting firm...")

consulting_status = hierarchical_agent.tool.agent_graph(
    action="status",
    graph_id="consulting_firm"
)

print(f"Consulting Firm Status: {consulting_status}")
print("============================================================")
print("============================================================")
# Manage the network
hierarchical_agent("Shut down the content consulting_firm when they're done")

### Natural Language Interface

The agent_graph tool also supports natural language interactions, making it easier to work with multi-agent systems conversationally. Let's see how it works with a `creative_writers` agent that has a star topology with a creative director, two writers and an editor. Architecture should look like this:


<div style="text-align:left">
    <img src="images/content_writer_architecture.png" width="55%" />
</div>

In [ ]:
# Step 1: Create a simple agent for natural language interactions
nl_agent = Agent(tools=[agent_graph])

print("Using natural language to create and manage agent networks...")

# Step 2: Create a team using natural language
nl_response = nl_agent(
    "Create a content creation team called 'creative_writers' with a creative director, two writers, and an editor. Use a star topology where the creative director coordinates everyone."
)

print(f"Natural Language Creation: {nl_response}")
print("============================================================")
print("============================================================")

# Step 3: Give the team a task
task_response = nl_agent(
    "Ask the creative_writers team to brainstorm ideas for a science fiction short story about AI and human collaboration in space exploration."
)

print(f"Creative Task Response: {task_response}")
print("============================================================")
print("============================================================")

# Step 4: Check on the team's progress
progress_response = nl_agent(
    "Show me the status of the creative_writers team and what they're working on."
)

print(f"Team Progress: {progress_response}")
print("============================================================")
print("============================================================")

nl_agent("Shut down the content creative_writers")


## Key Takeaways and Best Practices

### When to Use Each Topology:

**Star Topology:**
- Content creation with editorial oversight
- Customer service with escalation paths
- Research coordination with specialists
- Centralized decision-making processes

**Mesh Topology:**
- Collaborative problem-solving
- Consensus building and debates
- Peer review processes
- Democratic decision-making

**Hierarchical Topology:**
- Organizational structures
- Multi-level review processes
- Project management with delegation
- Layered data processing

### Best Practices:

1. **Clear Role Definition**: Each agent should have a specific, well-defined role
2. **Appropriate System Prompts**: Tailor prompts to the agent's expertise and position
3. **Proper Cleanup**: Always stop networks when they're no longer needed
4. **Status Monitoring**: Regularly check network status for debugging
5. **Strategic Communication**: Use targeted messaging for specific agents
6. **Resource Management**: Monitor active networks to avoid resource waste

## Conclusion

You've now mastered the fundamentals of building multi-agent systems with Strands Agent Graph! You can create sophisticated networks of specialized AI agents that collaborate to solve complex problems.

The key to successful multi-agent systems is:
- Matching topology to use case
- Defining clear agent roles and responsibilities  
- Establishing proper communication patterns
- Managing resources and cleanup effectively

From here, you can build increasingly sophisticated systems for real-world applications in research, content creation, decision-making, customer service, and beyond.